In [1]:
import tensorflow as tf

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train, x_test = x_train.astype('float32'), x_test.astype('float32')

x_train, x_test = x_train.reshape([-1, 784]), x_test.reshape([-1, 784])

x_train, x_test = x_train /255., x_test /255.

y_train, y_test = tf.one_hot(y_train, depth =10), tf.one_hot(y_test, depth=10)

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(60000).batch(100)
train_data_iter = iter(train_data)

In [22]:
class SoftmaxRegression(tf.keras.Model):
  def __init__(self):
    super(SoftmaxRegression, self).__init__()
    self.softmax_layer = tf.keras.layers.Dense(10,
                                              activation=None,
                                              kernel_initializer='zeros',
                                              bias_initializer='zeros')

  def call(self, x):
    logits = self.softmax_layer(x)

    return tf.nn.softmax(logits)

In [23]:
@tf.function
def cross_entropy_loss(y_pred, y):
  return tf.reduce_mean(-tf.reduce_sum(y * tf.math.log(y_pred), axis=[1]))

optimizer = tf.optimizers.SGD(0.5)

In [24]:
@tf.function
def train_step(model, x, y):
  with tf.GradientTape() as tape:
    y_pred =model(x)
    loss = cross_entropy_loss(y_pred, y)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

In [25]:
@tf.function
def compute_accuracy(y_pred, y):
  correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y,1))
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

  return accuracy

SoftmaxRegression_model = SoftmaxRegression()

In [26]:
for i in range(1000):
  batch_xs, batch_ys = next(train_data_iter)
  train_step(SoftmaxRegression_model, batch_xs, batch_ys)

In [27]:
print("정확도(Accuracy): %f" % compute_accuracy(SoftmaxRegression_model(x_test), y_test))

정확도(Accuracy): 0.916200
